# Model the Total Cost of Ownership for electric transport vehicles

### Defining our data set

Import libraries:

In [5]:
import os
import numpy as np
import pandas as pd
import xarray as xr
import cartopy.crs as ccrs
import matplotlib.pyplot as plt
import datetime
from scipy import spatial
%matplotlib inline

### Simple decision model for one customer traffic

We start by making an approximation of the costs for electric transportation of a one vehicle, one customer ride and the possibility of loading during the way and/or at the customer.

In [15]:
# set parameters
s1 = 10000; # distance to customer (m)
s2 = 7000; # distance from loading point to base (m)
s3 = 5000; # distance from loading point to customer (m)
m = 1; # number of rides per day
a_max = 20; # maximum action radius vehicle with a fully charged battery (kWh)
n_c = 3600; # waiting time at customer (sec)

In [16]:
# define total cost of ownership vehicle per km (euros/km)
def tco_km(p1, c_e, f, d, l, r = 0, c_t = 0):
    # p1 = initial costs vehicle and loading facilities (euros) -> is getting lower for newer vehicles f(year of purchase)
    # c_e = average costs for electricity (euros/kWh) -> depended on location and time, but for now assumed constant
    # f = average consumption (kWh/km)   
    # c_m = average maintenance costs per year (euros) -> needs to be a function of age f(year of purchase)
    # d = average distance covered per year (km)
    # l = economic life expectancy (years) -> is growing for newer electric vehicles
    # c_i = insurance costs per year (euros)
    # r = residual valu after l years of normal use (euros) -> now almost zero, but growing for newer vehicles f(year of purchase)
    # c_t = motor vehicle taxes per year (euros) -> zero until 2025, but after this growing
    
    depreciation = (p1 - r)/l;
    fixed_costs = (depreciation + c_i + c_t + c_m)/d;
    variable_costs = c_e * f;
    
    return fixed_costs + variable_costs

In [12]:
def calculate_ride_costs(d1,d2,d3,a_max,n_c,m1,m2,v,c_e,f,c_tco):
    # s1,s2,s3 = distance to customer, directly or via loading facility (m)
    # a_max = maximum capacity accu (kWh)
    # n_c = waiting time at customer (sec)
    # m1 = mass of loaded vehicle (kg)
    # m2 = mass of empty vehicle (kg)
    # v = average speed vehicle over total ride (m/s)
    # c_e = average costs for electricity (euros/kWh) -> depended on location and time, but for now assumed constant
    # f = average consumption (kWh/km)  
    # c_tco = average costs per km (euros/km)
        
    # check if input is correct
    if (d1 >= (d2 + d3)):
        print("input distances not correct: d1>=d2+d3");
        
    # calculate distance ride and total loading time
    
    # calculate driving costs
    driving_costs = 1/3600 * c_tco * s_tot * f/(m * v);
    
    # calculate loading costs
    loading costs = c_l * v_l * n_l + c_l * a_max;
    
    # calculate total costs        
    return  driving_costs + loading_costs;


### Complex model

To come up with a smart TCO model that includes the evaluation of the cost components in time, we need to define a time scale first.

In [7]:
time = pd.date_range("2020-01-01", freq="D", periods=365 * 10)

ds = xr.Dataset({"foo": ("time", np.arange(365 * 10)), "time": time})

We start by defining six different vehicle categories, defined by their average load capacity and fuel type, that exist at the moment.

N.B. All parameter values are averages and are taken from the TNO report and website below or guessed, for the sake of early testing of the model. In case of energy costs, the range for electric vehicles is very large and depends on the place of loading (work, home, on the road), loading duration, and also varies during the day. Thus loading time and costs are dependent parameters. 

[TNO, 2018]: ELEKTRISCHE BESTELAUTO’S IN NEDERLAND – MARKTONTWIKKELINGEN 2017-2025

https://autofans.be/nieuws/overheid/43275-dit-kost-100-km-rijden-motortype-diesel-benzine-cng-ev

In [ ]:
dict = {"car type": ["small van", "medium van", "large van", "small E-van", "medium E-van", "large E-van"],
       "weight (kg))": [1500, 2000, 2500],
       "maximum load (kg)": [700, 1000, 1500, 740, 1000, 1100],
       "initial cost (euros)": [],
       "energy type": ["diesel", "diesel", "diesel", "electric", "electric", "electric"],
       "energy cost/km (euros)": [0.077, 0.077, 0.077, 0.02, 0.02, 0.02],
       "range on full tank/battery (km)": [600, 600, 600, 150, 170, 200],
       "accu capacity (kWh)": [Nan, Nan, Nan, 20, 40, 60],
       "loading time (sec)": [23, 23, 23, 4380, 4380, 4380],
       "lifetime (years)": [9.3, 9.3, 9.3, ],
       "residual value": [200.4, 143.5, 1252, 1357, 52.98],
       "maintenance cost/km": []
    }